In [24]:
import utils.dataset_functions as df
import utils.user_features as uf
import utils.two_towers as ttn
import torch
import numpy as np
from pathlib import Path
import pandas as pd
import faiss

# Dimensions of the tower FFN
output_dim      = 16
hidden_dim      = 256
id_dim          = 16


# Training
num_epochs      = 50
learning_rate   = 1e-3
batch_size      = 256
patience        = 5
device='cuda' if torch.cuda.is_available() else 'cpu'
print("Training models on:", device)

Training models on: cuda


### Recommendations & Evaluation
Ik weet niet precies hoe alles precies in elkaar steekt dus ik schrijf ff een skelet en dan moet je maar ff kijken of dingen kloppen.

Dit zijn de stappen die ik probeer te zetten:
- Lijst van nummers inladen
- Lijst van users inladen
- modellen inladen
- per model
  - bereken song embeddings (yv)
  - bereken user embeddings (uv)
  - maak index voor reccomendations
  - per user
    - maak recommendations
    - voeg recommendations en test set (van die user) samen
    - reken dot-product uit per query recommendation pair
    - rerank op dot-product
    - beoordeel rankings (op reinforcement signal / like-dislikes



Per user

1. eerste sample uit de testset wordt gebruikt
2. De rest vd samples zijn voor de ranking

In [ ]:
user_item_data = pd.read_csv(Path("dataset") / "processed" / "merged.csv", index_col=False).drop(["normalized_embed"], axis=1)
songs = torch.load(Path("dataset") / "processed" / "song_subset.pt")
binary_embeddings = songs['binary']
non_binary_embeddings = songs['continueous']

In [28]:
already_seen = {} # user: [num_ids of songs already listened to]

for u in user_item_data['uid'].unique():
    a = user_item_data[(user_item_data['uid'] == u)]
    already_seen[u] = a[0:int(0.7 * len(a))]['item_id'].unique().tolist()


In [ ]:
binary_model = model = ttn.DualAugmentedTwoTower("binary_label", hidden_dim, output_dim, id_dim)
binary_model.load(Path("models")/ "binary_label.pt")
non_binary_model = ttn.DualAugmentedTwoTower("continueous_label", hidden_dim, output_dim, id_dim)
non_binary_model.load(Path("models")/ "continueous_label.pt")

test_set = df.load_tensor_dataloader("val", Path("dataset")/"processed", 1, label_id=2)



# Create user embeddings
# (Ik weet ff niet hoe de columns heten, dubbelcheck dit ff)

users = {}
for user_features, song_embedding, label, interaction, user_id, song_id in test_set:
    if song_id.item() not in already_seen[user_id.item()]:
        if users.get(user_id.item()) is None:
            if len(users) > 5:
                break
            users[user_id.item()] = {}

            users[user_id.item()]['user_embed_bin'] = binary_model.user_pass(user_features, user_id).detach().squeeze(0)
            users[user_id.item()]['user_embed_nbin'] = non_binary_model.user_pass(user_features, user_id).detach().squeeze(0)

            users[user_id.item()]['user_interact_ratio'] = user_features.squeeze(0)[4].item()
            users[user_id.item()]['song_data'] = {}

        
        else:
            users[user_id.item()]['song_data'][song_id.item()] = (label.squeeze(0).tolist(), interaction.item())

    
        
    
recommendations = {}
non_binary_index = np.array([i.tolist() for i in non_binary_embeddings.values()])
binary_index = np.array([i.tolist() for i in binary_embeddings.values()])

print(len(non_binary_index))
non_binary_model.create_index(non_binary_index)
binary_model.create_index(binary_index)
for u in users.keys():
    
    
    user = users[u]
    
    

    nbin_query = user["user_embed_nbin"][np.newaxis, :]
    nbin_distances, nbin_indices = non_binary_model.recommendations(nbin_query, 100)

    bin_query = user["user_embed_bin"][np.newaxis, :]
    bin_distances, bin_indices = binary_model.recommendations(bin_query, 100)
    
    
    nbin_score = [(nbin_query * non_binary_embeddings[e]).sum() for e in nbin_indices[0]]
    bin_score = [(bin_query * binary_embeddings[e]).sum() for e in bin_indices[0]]


    


    recommendations[u] = {
        "non_binary": (nbin_score, nbin_indices, nbin_distances),
        "binary": (bin_score, bin_indices, bin_distances)
    }

    

    # TODO
    # 1. calculate scores for all the test set songs bin and nbin
    # 2. Create dataframe of the output.
    # 3. Do the metrics things

    # indices, distance = binary_model.recommendations(user['user_embed_bin'], 25)

    # users['yu'] = users['user_features', 'user_id'].apply(lambda user: model.user_pass(user['user_features'], user['user_id']), axis = 1)
    
    # # Create index for reccommendations
    # model.create_index(songs['ys'].to_numpy())

    # # Metric voor alle recommendations
    # RS_scores = []  # Reinforcement Signal
    # LDL_scores = [] # Like-DisLike

    # # Loop users
    # for index, user in users.iterrows:

    #     # Get user embedding
    #     query = user['yu']

    #     # Make 25 recommendations
    #     indices, distances = model.recommendations(query, 25)
    #     recommendations = songs.iloc[indices]

    #     # Merge recommendations with the users test-split
        
    #     # recommendations = ... (add test songs here)

    #     # Calculate scores (dot product with query)
    #     recommendations['score'] = recommendations['ys'].apply(lambda yv: (query * yv).sum())

    #     # Re-rank
    #     ranking = recommendations.sort_values(by = 'score')

    #     # Evaluate ranking
    #     # Weet nog niet hoe. We moeten iig de like/dislikes mergen met de ranking en daar iets van een score uit halen
    #     # Ook kunnen we nog de reinforcement signal mergen en daar Froukjes metrics op los laten
        
    #     # RS_score = ...
    #     # LDL_score = ...

    #     # RS_scores.append(RS_score)
    #     # LDL_scores.append(LDL_score)

    # # Aggregate scores (average?)
    # RS_score = np.mean(RS_scores)
    # LDL_score = np.mean(LDL_scores)
    # print(f'{model_name}: RS = {RS_score}, LDL = {LDL_score}')

3192
